In [2]:
! unzip ../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip 
! unzip ../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip 
! unzip ../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip 
! unzip ../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip

Archive:  ../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
Archive:  ../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
replace test_labels.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
Archive:  ../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
Archive:  ../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [3]:
! pip install transformers

In [4]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import transformers
from tqdm.notebook import tqdm

In [5]:
bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased") 
bert_model.trainable = False

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

2022-01-20 08:13:09.084732: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-20 08:13:09.085934: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-20 08:13:09.086720: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-20 08:13:09.087599: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [6]:
batch_size=32
max_len=128
EPOCHS=2

In [7]:
data=pd.read_csv("./train.csv",nrows=75000)
test_labels=pd.read_csv("./test_labels.csv",nrows=75000)
# train_data.head() 
# print(data.loc[0])
# data.columns

# data["toxic"].value_counts()
data.head()
test_labels.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [8]:
row_no=list(data.shape)[0]
for i in range(row_no):
    row=data.loc[i]
    row["comment_text"]=re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", row["comment_text"])

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [9]:
# x_train[0]
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [10]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
# [[0.09499235 0.09728582 0.20707038 0.2026455  0.18660283 0.21140313]]
labels =  test_labels[label_cols].values

In [11]:
# indexes = np.arange(12)
# np.array(labels[indexes], dtype="int32")
labels

array([[-1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1],
       ...,
       [-1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1],
       [ 0,  0,  0,  0,  0,  0]])

In [12]:
from sklearn.model_selection import train_test_split 
input_sen=data["comment_text"].values
# print(input_sen)
train_inputs,validation_inputs,train_labels,validation_labels=train_test_split(input_sen,labels,random_state=0,test_size=0.2) 


print(train_inputs.shape)
print(train_labels.shape)

print(validation_inputs.shape)
print(validation_labels.shape)
 

(60000,)
(60000, 6)
(15000,)
(15000, 6)


In [20]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence): 
    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
         
        
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=128,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )   

        bert_output = bert_model(**encoded)
        
        sequence_output = bert_output.last_hidden_state
#         pooled_output = bert_output.pooler_output
         
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return sequence_output, labels
        else:
            return sequence_output

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [21]:
train_dataset=BertSemanticDataGenerator(train_inputs,train_labels,shuffle=True)
validation_dataset=BertSemanticDataGenerator(validation_inputs,validation_labels,shuffle=False)

In [15]:
# for d in tqdm(train_dataset):
#     print(d)

In [29]:
# Sequence model
strategy = tf.distribute.MirroredStrategy()

with strategy.scope(): 
    input_layer = tf.keras.layers.Input(shape=(128, 768), name=None)
#     input_layer=tf.reshape(input_layer, (128, 768), name=None) 
    
    bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(input_layer) 
    
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.1)(concat)    
    output = tf.keras.layers.Dense(6, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=input_layer, outputs=output
    )
    
    model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128, 768)]   0                                            
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 128, 128)     426496      input_4[0][0]                    
__________________________________________________________________________________________________
global_average_pooling1d_3 (Glo (None, 128)          0           bidirectional_2[0][0]            
__________________________________________________________________________________________________
global_max_pooling1d_3 (GlobalM (None, 128)          0           bidirectional_2[0][0]            
____________________________________________________________________________________________

In [30]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
)

In [31]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=2
)

2022-01-20 08:34:28.140379: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_5115392"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_INT32
    }
  }
}
. Consider either turning off auto

Epoch 1/2
1875/1875 [==============================] - ETA: 0s - loss: -6.6213 - acc: 0.4993

2022-01-20 08:42:13.992122: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_7840452"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_INT32
    }
  }
}
. Consider either turning off auto

1875/1875 [==============================] - 579s 307ms/step - loss: -6.6213 - acc: 0.4993 - val_loss: -6.3793 - val_acc: 0.9992
Epoch 2/2
1875/1875 [==============================] - 573s 306ms/step - loss: -6.6176 - acc: 0.5053 - val_loss: -6.3483 - val_acc: 0.9992


In [34]:
s4 = 'fuck'
sentence_pairs = np.array([s4])
test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

pro=model.predict(test_data)
print(pro)
print(label_cols[np.argmax(pro)])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
2022-01-20 08:55:42.592112: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_11926300"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
       

[[0.2571675  0.21692714 0.16095094 0.0555994  0.1521901  0.15716496]]
toxic


In [35]:
model.save("my_model.h5")

In [ ]:
# from keras.preprocessing.sequence import pad_sequences
# # to convert the iput ids array in same size(column no=max(column no))
# tokenizer=transformers.BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
# max_length=128
# bert_model=transformers.TFBertModel.from_pretrained("bert-base-uncased")
# bert_model.trainable=False

# def tokenize(data,tokenizer=tokenizer,max_length=max_length):
# #     input_ids=[]
# #     attention_masks=[]
#     bert_outputs=[]
#     for sentence in tqdm(data):
        
#         encoded_data=tokenizer.batch_encode_plus(
#                         sentence,
#                         add_special_tokens=True,
#                         max_length=max_length,
#                         truncation=True,
#                         return_attention_mask=True,
#                         return_token_type_ids=True,
#                         pad_to_max_length=True,
#                         return_tensors="tf",
#                     )

# #         input_id=np.array(encoded_data["input_ids"],dtype="int32")
# #         attention_mask=np.array(encoded_data["attention_mask"],dtype="int32") 
        
#         bert_output=bert_model(**encoded_data)
#         sequence_output = bert_output.last_hidden_state
#         bert_outputs.append(sequence_output)
#     return bert_outputs
# #         input_ids.append(input_id)
# #         attention_masks.append(attention_mask)
        
# #     return [input_ids,attention_masks]

# bert_op=tokenize(data['comment_text'])
# # input_ids=pad_sequences(bert_op[0],maxlen=max_length,dtype='long',value=0,truncating="post",padding="post")
# # attention_masks=bert_op[1]
# # bert_op.shape

In [ ]:
# len(bert_op)

In [ ]:
# attention_masks=np.array(attention_masks)
# attention_masks=pad_sequences(attention_masks,maxlen=max_length,dtype='long',value=0,truncating="post",padding="post")
# input_ids.shape
# attention_masks.shape

In [ ]:
# # creating batched dataset
# epochs=2
# def create_batch_dataset(data,epochs=epochs,batch_size=batch_size,buffer_size=1000,train=True):
#     dataset=tf.data.Dataset.from_tensor_slices(data)
# #     print(dataset.as_numpy_iterator())
#     if train:
#         dataset=dataset.shuffle(buffer_size=buffer_size)
#         # uses for shuffling the dataset.Select the first buffer_size element from dataset
#     dataset=dataset.repeat(epochs)
#     # just repeat the whole dataset
#     dataset=dataset.batch(batch_size=batch_size)
#     # devide the whole dataset into batch size and create an array of array
#     if train:
#         dataset=dataset.prefetch(1)
#     #     It has no concept of examples vs. batches. examples.prefetch(2) will prefetch two 
#     # elements (2 examples), while examples.batch(20).prefetch(2) will prefetch 2 elements (2 
#     # batches, of 20 examples each).
#     return dataset
# train_dataset=create_batch_dataset((train_inputs,train_masks,train_labels),train=True)
# validation_dataset=create_batch_dataset((validation_inputs,validation_masks,validation_labels),train=True)

In [ ]:
# print(train_dataset.as_numpy_iterator())
# print(validation_dataset.as_numpy_iterator())

In [ ]:
# list(train_dataset.as_numpy_iterator())[0][0].shape

In [ ]:
# for i, (token_ids, masks, labels) in enumerate(tqdm(train_dataset)):
#             print(token_ids.shape)

In [ ]:
# https://www.kaggle.com/nkaenzig/bert-tensorflow-2-huggingface-transformers
# https://www.kaggle.com/satyamkryadav/bert-model-96-77/notebook